# Data: Dashboard Compilation

This notebook compiles the information required to build the dashboard of each metropolitan area.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import dashboard_functions
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image
import os
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
df = pd.read_csv('../data/interim/census/household_shopping_concat.csv')

In [3]:
df_summary = pd.read_csv('../data/processed/shopping_statistics_msa.csv')

In [4]:
df['GENERATION'] = df.TBIRTH_YEAR.apply(dashboard_functions.generation_)

In [5]:
df_summary.rename({'CHNGHOW4': 'RFID-NFC', 
                   'CHNGHOW5': 'CASH', 
                   'CHNGHOW6': 'AVOID RESTR',
                   'CHNGHOW7': 'RESUME RESTR'}, axis=1, inplace=True)

In [6]:
MSA_codes = list(df_summary.EST_MSA)
MSA_names = list(df_summary.MSA_NAME)

In [7]:
df = df[df.EST_MSA.isin(MSA_codes)]

### Correlation between Consumer variables

In [8]:
df1 = df.loc[:, ['EST_MSA', 'CHNGHOW1', 'CHNGHOW2', 'CHNGHOW3', 'CHNGHOW4', 
                'CHNGHOW5', 'CHNGHOW6', 'CHNGHOW7']]

In [9]:
df1.rename({'CHNGHOW1': 'ONLINE',
           'CHNGHOW2': 'PICK-UP',
           'CHNGHOW3': 'IN-STORE',
           'CHNGHOW4': 'RFID-NFC', 
           'CHNGHOW5': 'CASH', 
           'CHNGHOW6': 'AVOID RESTR',
           'CHNGHOW7': 'RESUME RESTR'}, axis=1, inplace=True)

In [10]:
# for index in range(len(MSA_codes)):
#     dashboard_functions.shopping_behaviors_correlation(df1, MSA_codes[index], MSA_names[index])

### Each Consumer variable against demographics

In [11]:
# shopping_variables = ['CHNGHOW1', 'CHNGHOW2', 'CHNGHOW3', 'CHNGHOW4', 'CHNGHOW5', 'CHNGHOW6', 'CHNGHOW7']
# name_shopping_variables = ['ONLINE','PICK-UP','IN-STORE','RFID-NFC','CASH','AVOID RESTR','RESUME RESTR']

# for i in range(len(MSA_codes)):
#     for j in range(len(shopping_variables)):
#         dashboard_functions.cramers_matrix(df, MSA_codes[i], 
#                                            MSA_names[i], 
#                                            shopping_variables[j], 
#                                            name_shopping_variables[j])

## Section 2: Mobility Trends

In [12]:
df_descartes_counties = pd.read_csv('../data/interim/mobility/m50_max_counties.csv')
df_descartes_counties_percent = pd.read_csv('../data/interim/mobility/m50_percent_counties.csv')

df_apple = pd.read_csv('../data/interim/mobility/apple_mobility_cities.csv', index_col=0)

df_foursquare_dma = pd.read_csv('../data/interim/mobility/foursquare_dma.csv', index_col=0)
df_google = pd.read_csv('../data/interim/mobility/google_counties.csv', index_col=0)

In [13]:
MSA_names

['Atlanta-Sandy Springs-Alpharetta',
 'Boston-Cambridge-Newton',
 'Chicago-Naperville-Elgin',
 'Dallas-Fort Worth-Arlington',
 'Detroit-Warren-Dearborn',
 'Houston-The Woodlands-Sugar Land',
 'Los Angeles-Long Beach-Anaheim',
 'Miami-Fort Lauderdale-Pompano Beach',
 'New York-Newark-Jersey City',
 'Philadelphia-Camden-Wilmington',
 'Phoenix-Mesa-Chandler',
 'Riverside-San Bernardino-Ontario',
 'San Francisco-Oakland-Berkeley',
 'Seattle-Tacoma-Bellevue',
 'Washington-Arlington-Alexandria']

In [14]:
def indexed_mobility_average_person(df, location):
    df_index = df[df.NAME == location].loc[:, '2020-03-01':]

    df_index = pd.melt(df_index, var_name='date', value_name='value')
    df_index['date'] = pd.to_datetime(df_index['date'])
    df_index['location'] = location
    df_index['src'] = 'indexed regular member'
    
    return df_index.loc[:, ['date', 'location', 'src', 'value']]

def mobility_average_person(df, location):
    df_kms= df[df.NAME == location].loc[:, '2020-03-01':]

    df_kms = pd.melt(df_kms, var_name='date', value_name='value')
    df_kms['date'] = pd.to_datetime(df_kms['date'])
    df_kms['location'] = location   
    df_kms['src'] = 'kms regular member'
    
    return df_kms.loc[:, ['date', 'location', 'src', 'value']]

In [15]:
# dict_locations = {'Miami-Fort Lauderdale-Pompano Beach': ['Miami-Dade County', 'Broward County', 'Palm Beach County'],
#                   'San Francisco-Oakland-Berkeley': ['San Francisco County', 'Alameda County', 'Marin County', 
#                                                      'Contra Costa County', 'San Mateo County'], 
#                   'Dallas-Fort Worth-Arlington': ['Collin County', 'Dallas County', 'Denton County', 'Ellis County', 
#                                                   'Hunt County', 'Kaufman County', 'Rockwall County', 'Johnson County',
#                                                   'Parker County','Tarrant County','Wise County']
#                   'Los Angeles-Long Beach-Anaheim': ['Ventura County', 'San Bernardino County', 'Riverside County',
#                                                      'Los Angeles County', 'Orange County'],
#                   'Atlanta-Sandy Springs-Alpharetta': [],
#                   'Boston-Cambridge-Newton': [],
#                   'Chicago-Naperville-Elgin': [],
#                   'Detroit-Warren-Dearborn': [],
#                   'Houston-The Woodlands-Sugar Land': [],
#                   'Los Angeles-Long Beach-Anaheim': [],
#                   'New York-Newark-Jersey City': [],
#                   'Philadelphia-Camden-Wilmington': [],
#                   'Phoenix-Mesa-Chandler': [],
#                   'Riverside-San Bernardino-Ontario': [],
#                   'Seattle-Tacoma-Bellevue': [],
#                   'Washington-Arlington-Alexandria': []

#                  }

In [16]:
locations = ['Miami-Dade County',
             'San Francisco County']
df_index = pd.DataFrame()
df_kms = pd.DataFrame()

for i, location in enumerate(locations):
    temp1 = indexed_mobility_average_person(df_descartes_counties, location)
    temp2  = mobility_average_person(df_descartes_counties_percent, location)
    if i == 0:
        df_index = temp1
        df_kms = temp2
    else:
        df_index = pd.concat([df_index, temp1], ignore_index=True)
        df_kms = pd.concat([df_kms, temp2], ignore_index=True)

In [17]:
def mobility_venues_foursquare(df, location):
    #Locations: Los Angeles, New York, SanFrancisco-Oakland-SanJose, Seattle-Tacoma
    try:
        df = df[[location,'class']]
    except:
        print('this location does not exist. Select one between the following list: \n1. Los Angeles \n2. New York \n3. SanFrancisco-Oakland-SanJose \n4. Seattle-Tacoma')
        return None
    df.index = pd.to_datetime(df.index)
    
    temp = df.groupby('class').count().reset_index()
    max_data = max(temp[location])
    #only display historical data with 80% of the longer serie
    valid_venues = list(temp[temp[location] >= 0.8*max_data]['class'])
    df_output = df[df['class'].isin(valid_venues)].reset_index().rename({' ':'date'}, axis=1)
    df_output['location'] = location
    df_output['source'] = 'venues foursquare'
    df_output['value'] = df_output[location]

    return df_output.loc[:, ['date', 'location', 'source', 'value', 'class']]

def mobility_venues_google(df, location):
    df = df[df.NAME == location]
    df1 = pd.melt(df, id_vars=['NAME', 'category'], var_name='Date')
    df1.Date = pd.to_datetime(df1.Date)
    temp = df1.groupby('category').count().reset_index()

    max_data = max(temp['value'])
    #only display historical data with 80% of the longer serie
    valid_venues = list(temp[temp['value'] >= 0.8*max_data]['category'])
    
    df_output = df1[df1['category'].isin(valid_venues)].reset_index().rename({'Date':'date', 'category': 'class'}, axis=1)
    df_output.drop(columns={'index'}, inplace=True) 
    df_output['location'] = location
    df_output['source'] = 'venues google'
    
    return df_output.loc[:, ['date', 'location', 'source', 'value', 'class']]

In [18]:
locations = ['Miami-Dade County', 'San Francisco County', ]
venues_google = pd.DataFrame()

for i, location in enumerate(locations):
    temp1 = mobility_venues_google(df_google, location)
    if i == 0:
        venues_google = temp1
    else:
        venues_google = pd.concat([venues_google, temp1], ignore_index=True)
        
locations = ['SanFrancisco-Oakland-SanJose']
venues_foursquare = pd.DataFrame()

for i, location in enumerate(locations):
    temp2  = mobility_venues_foursquare(df_foursquare_dma, location)
    if i == 0:
        venues_foursquare = temp2
    else:
        venues_foursquare = pd.concat([venues_foursquare, temp2], ignore_index=True)

In [19]:
def serie_decomposition(df, colname, model, display, location):

    df = df[df.city == location]
    df.date = pd.to_datetime(df.date)
    df.set_index('date', inplace=True)

    if df[colname].isnull().any():
        df.fillna(method='ffill', inplace=True)
    result_add = seasonal_decompose(df[colname], model=model, extrapolate_trend='freq')
    df_reconstructed = pd.concat([result_add.seasonal, result_add.trend, result_add.resid, result_add.observed], axis=1)
    df_reconstructed.columns = ['seas', 'trend', 'resid', 'actual_values']
        
    return df_reconstructed

In [20]:
df_apple.city.unique()

array(['New York City', 'Los Angeles', 'Houston', 'Detroit', 'Dallas',
       'Boston', 'Philadelphia', 'Miami', 'Chicago', 'San Diego',
       'Seattle', 'San Francisco - Bay Area'], dtype=object)

In [21]:
def decomposition_selection(df, signal):
    if signal in ['seas', 'trend', 'resid', 'actual_values']:
        pass
    else:
        return None
    df_transit = pd.DataFrame()
    df_driving = pd.DataFrame()
    df_walking = pd.DataFrame()

    for i, location in enumerate(list(df.city.unique())):
        out1 = serie_decomposition(df, 'trn', 'additive', False, location)
        out2 = serie_decomposition(df, 'drv', 'additive', False, location)
        out3 = serie_decomposition(df, 'wlk', 'additive', False, location)

        out1 = out1.reset_index()
        out1['src'] = 'transit '+ signal
        out1['location'] = location
        out1['value'] = out1[signal]

        out2 = out2.reset_index()
        out2['src'] = 'driving '+ signal
        out2['location'] = location
        out2['value'] = out2[signal]

        out3 = out3.reset_index()
        out3['src'] = 'walking '+ signal
        out3['location'] = location
        out3['value'] = out3[signal]

        if i == 0:
            df_transit = out1
            df_driving = out2
            df_walking = out3
        else:
            df_transit = pd.concat([df_transit, out1], ignore_index=True)
            df_driving = pd.concat([df_driving, out2], ignore_index=True)
            df_walking = pd.concat([df_walking, out3], ignore_index=True)
            
            
    df_transit = df_transit.loc[:, ['date', 'location', 'src', 'value']]
    df_driving = df_driving.loc[:, ['date', 'location', 'src', 'value']]
    df_walking = df_walking.loc[:, ['date', 'location', 'src', 'value']]
            
    return df_transit, df_driving, df_walking

In [22]:
df_trn = pd.DataFrame()
df_drn = pd.DataFrame()
df_wlk = pd.DataFrame()

for signal in ['seas', 'trend', 'resid', 'actual_values']:
    trn, drv, wlk = decomposition_selection(df_apple, signal)
    if i == 0:
        df_trn = trn
        df_drn = drv
        df_wlk = wlk
    else:
        df_trn = pd.concat([df_trn, trn], ignore_index=True)
        df_drn = pd.concat([df_drn, drv], ignore_index=True)
        df_wlk = pd.concat([df_wlk, wlk], ignore_index=True)

In [23]:
df_mobility_indexed = pd.concat([df_index, df_kms, df_trn, df_drn, df_wlk])

In [24]:
df_mobility_indexed.to_csv('../data/processed/dashboard_mobility_indexed.csv', index=False)

In [26]:
df_foot_traffic = pd.concat([venues_foursquare, venues_google])
df_foot_traffic.to_csv('../data/processed/dashboard_foot_traffic.csv', index=False)

## Section 3: Restaurants

The following analysis includes 50 businesses/every zip code of San Francisco county. The information of the restaurants was extracted using FUSION API Yelp, which allows a maximum of 50 results for endpoint using the zip codes as keywords and words as Restaurant.

In [36]:
df_restaurants_sf = pd.read_csv('../data/interim/restaurants/yelp_sf.csv')
df_restaurants_miami = pd.read_csv('../data/interim/restaurants/yelp_miami.csv')
df_restaurants_dallas = pd.read_csv('../data/interim/restaurants/yelp_dallas.csv')
df_restaurants_la = pd.read_csv('../data/interim/restaurants/yelp_losAngeles.csv')

In [37]:
list_df_restaurants = [df_restaurants_sf, 
                       df_restaurants_miami, 
                       df_restaurants_dallas,
                       df_restaurants_la]

In [38]:
df_restaurants = pd.DataFrame()

for i, df in enumerate(list_df_restaurants):
    try:
        df.drop(columns=['Unnamed: 0'], inplace=True)
    except:
        pass
    df['purchase'] = df['transactions'].apply(dashboard_functions.transaction_cleaning)
    df['pickup_delivery'] = df['purchase'].apply(dashboard_functions.check_pickup_delivery)
    if i == 0:
        df_restaurants = df
    else:
        df_restaurants = pd.concat([df_restaurants, df], ignore_index=True)

In [39]:
df_restaurants.to_csv('../data/processed/dashboard_yelp_restaurants.csv', index=False)